# Circuit Generation

Construct a memory experiment circuit and inspect the output.


### ErrorModel and CircuitBuildOptions in Circuit Generation

`get_cardinal_circuit(...)` now takes structured objects instead of four separate noise scalars.

- `ErrorModel(idle_error, sqgate_error, tqgate_error, spam_error)` defines the noise channels.
- `CircuitBuildOptions(get_all_detectors, noisy_zeroth_round, noisy_final_meas)` defines circuit-generation behavior.

Tuple support for asymmetric Pauli channels:

- `idle_error` and `sqgate_error`: `(px, py, pz)` for `DEPOLARIZE1`
- `tqgate_error`: 15-tuple ordered as
  `(IX, IY, IZ, XI, XX, XY, XZ, YI, YX, YY, YZ, ZI, ZX, ZY, ZZ)` for `DEPOLARIZE2`


In [1]:
import numpy as np
import stim

from quits.circuit import check_overlapping_CX
from quits import ErrorModel, CircuitBuildOptions, get_cardinal_circuit
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


In [2]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 10
num_trials = 100
W, F = 5, 3
max_iter, osd_order = 10, 1


In [3]:
# Define structured noise model and circuit-construction options
error_model = ErrorModel(
    idle_error=p,
    sqgate_error=p,
    tqgate_error=p,
    spam_error=p,
)

circuit_build_options = CircuitBuildOptions(
    get_all_detectors=False,
    noisy_zeroth_round=True,
    noisy_final_meas=False,
)


In [4]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'same_logicals_ZX_anticommute': True, 'different_logicals_ZX_commute': True, 'all_tests_passed': True}


In [5]:
# Generate memory experiment circuit
circuit = stim.Circuit(
    get_cardinal_circuit(code, error_model=error_model, num_rounds=num_rounds, basis="Z", circuit_build_options=circuit_build_options)
)
check_overlapping_CX(circuit, verbose=True);
# print(circuit)


No overlapping CX gates found.
